In [1]:
import tensorflow as tf
import contextlib2
import json
import logging
import sys
import os
sys.path.append('..')
from mask_rcnn.anchors import Anchors, AnchorLabeler
from mask_rcnn.hyperparameters.dataset_params import get_data_params
from mask_rcnn import dataset_utils
import matplotlib.pyplot as plt
from pycocotools import mask
import numpy as np
from tqdm.notebook import tqdm

In [9]:
logger = tf.get_logger()
logger.setLevel(logging.INFO)
train_dir = '/workspace/shared_workspace/data/coco/train2017/'
train_annotations_file = '/workspace/shared_workspace/data/coco/annotations/instances_train2017.json'
dataset_params = get_data_params()
dataset_params['image_size'] = (832, 1344)
output_path = '/workspace/shared_workspace/data/coco/tf_record/train-coco'
shards = 1024

In [5]:
images, category_index, annotations_index = dataset_utils.parse_annotations(train_annotations_file)
image_preprocess = dataset_utils.PreprocessDataset(dataset_params, train_dir)

In [10]:
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = dataset_utils.open_sharded_output_tfrecords(
        tf_record_close_stack, output_path, shards)
    images_count = 0
    for idx in tqdm(range(len(images))):
        example, shape = dataset_utils.create_record(idx, images, 
                                                     annotations_index, image_preprocess)
        if example:
            shard_idx = images_count % shards
            output_tfrecords[shard_idx].write(example.SerializeToString())
            images_count += 1